<a href="https://colab.research.google.com/github/shramee/snark-py/blob/main/snark_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: install galois

!pip install galois py_ecc


## R1CS and witness

R1CS comprises of three matrices describing all constraints between all signals (values). Mapping relationship between inputs, outputs and intermediate values.

Three matrices are `L`, `R` and `O` such that with witness `w`,

```
Lw  *  Rw  =  Ow
```

In [ ]:
import galois
import numpy as np

p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
# p = 71
FP = galois.GF(p)

# input arguments
x = FP(2)
y = FP(3)

v1 = x * x
v2 = y * y
v3 = 5 * x * v1
v4 = 4 * v1 * v2
out = 5*x**3 - 4*x**2*y**2 + 13*x*y**2 + x**2 - 10*y

w = FP([1, out, x, y, v1, v2, v3, v4])

print("w\n", w)

R = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 5, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 4, 0, 0, 0],
         [0, 0, 13, 0, 0, 0, 0, 0]])

L = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0]])

O = FP([[0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 1],
         [0, 1, 0, 10, FP(p - 1), 0, FP(p - 1), 1]])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)

print("Lw\n", Lw)
print("Rw\n", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw\n", LwRw)

print("Ow\n", Ow)

assert np.all(LwRw == Ow)

w
 [  1 104   2   3   4   9  40 144]
Lw
 [2 3 4 9 9]
Rw
 [ 2  3 10 16 26]
Lw * Rw
 [  4   9  40 144 234]
Ow
 [  4   9  40 144 234]


## QAP

We want to get to a polynomial field from from the R1CS vector field.

We want,

```
From Vector field,
    L  *  R   =  O
To polynomials
  l(x) * r(x) = o(x)
```

In [13]:
mtxs = [L, R, O]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = FP(np.zeros(m.shape[0], dtype=int))
        points_y = FP(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = FP(j+1)
            points_y[j] = m[j][i]

        poly = galois.lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)

    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

print(f'''
------------------------------------------
Lp
{Lp}
''')

print(f'''
------------------------------------------
Rp
{Rp}
''')

print(f'''
------------------------------------------
Op
{Op}
''')


------------------------------------------
Lp
[[                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            5
   91201011965996980092693357271905312868951518335066809765409184110732

Mutiplying (dot product) polynomial matrices with witness `w` yields coefficients for the polynomials

In [8]:
U = galois.Poly((w @ Lp)[::-1])
V = galois.Poly((w @ Rp)[::-1])
W = galois.Poly((w @ Op)[::-1])

print("U = ", U)
print("V = ", V)
print("W = ", W)

U =  11856131555579607412050136445347690672963697383558685269503193934395229601792x^4 + 20064222632519335620392538599819168831169334033714698148390020504361157787655x^3 + 20976232752179305421319472172538221959858849217065366246044112345468483141610x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955812x + 21888242871839275222246405745257275088548364400416034343698204186575808495601
V =  10944121435919637611123202872628637544274182200208017171849102093287904247809x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301415930x^3 + 10944121435919637611123202872628637544274182200208017171849102093287904247836x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079635x + 26
W =  12768141675239577212977070018066743801653212566909353367157285775502554955771x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602831936x^3 + 9120101196599698009269335727190531286895151833506680976540918411073253539611x^2 

Testing the polynomial with random tau

In [18]:
tau = FP(20)

assert U(tau)*V(tau) == W(tau) # this will fail


AssertionError: ignored

The assertion above fails because,

`U(tau) * V(tau)` is a polynomial with double the the degree of W (because of multiplication). So W needs to be balanced.

First we find target polynomial `T` that returns 0 for coordinates 1, 2, 3, 4 and 5.

The final equation looks like this,

```
U(x) * V(x)  =  W(x) + T(x) * H(x)
```

So
```
        U(x) * V(x) - W(x)
H(x) =  ––––––––––––––––––
               T(x)
```

In [15]:
# Prepare T
T = galois.Poly([1, p-1], field=FP)
for i in range(2, L.shape[0] + 1):
    T *= galois.Poly([1, p-i], field=FP)

# Calculate H
H = (U * V - W) // T

Now to assert with equation balanced,

In [20]:
tau = FP(20)

assert U(tau)*V(tau) == W(tau) + H(tau) * T(tau)


In [29]:
## [TOY] Trusted setup

In [25]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
tau = FP(20)

print(f"τ = {tau}")

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

# ...
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[τ^0], G2[τ^1], ..., G2[τ^d]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[T(τ)]G1 = {[normalize(point) for point in target_G1]}")

print(f"\n[τ]G2 = {[normalize(point) for point in tau_G2]}")

Setup phase
----------
Toxic waste:
τ = 20

T(τ) = 1395360
Trusted setup:
----------
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677), (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412)]
[T(τ)]G1 = [(1641247283492879903468444805169804215277964208279225379119694118848884481979, 18733245328562972535068072505811612803902036184072974454572123739995203670419), (1282524939337382483469274134339301655621870987447481453798307725808867601190, 1553918201

## [TOY] Proof generation

In [27]:
def evaluate_poly(poly, trusted_points, verbose=False):
    coeff = poly.coefficients()[::-1]

    assert len(coeff) == len(trusted_points), "Polynomial degree mismatch!"

    if verbose:
        [print(normalize(point)) for point in trusted_points]

    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    if verbose:
        print("-"*10)
        print(normalize(evaluation))
    return evaluation

print("\nProof generation:")
print("-"*10)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
B_G1 = evaluate_poly(V, tau_G1)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
Cw_G1 = evaluate_poly(W, tau_G1)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)

C_G1 = add(Cw_G1, HT_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")


Proof generation:
----------
[A]G1 = (19092006581455788758709004813424108450475230671546198110182704126760952021248, 18428185916649502171614192229986655674799279684527591370328182794110727996633)
[B]G2 = ((1110332524507442648511549408896049077062269578877062826069065960274388112308, 15815785354885964222010325771656100864105333417560377595802485750386873282739), (20784382045877636010618629654573620888044404319093695781168988411617616204166, 5234804291052944426941184034424257962428641145809086397589880058685491457835))
[C]G1 = (21755526246297599392782387322262927251662305599666002632514868138515690603377, 19883332083442129478217826420060112230198011363938980948134718366700920887106)


## [TOY] Verification

In [28]:
print("\nProof verification:")
print("-"*10)
# e(A, B) == e(C, G2[1])
assert pairing(B_G2, A_G1) == pairing(G2, C_G1), "Pairing check failed!"
print("Pairing check passed!")



Proof verification:
----------
Pairing check passed!
